In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [3]:
def get_text(text_list):
    arr = []
    for t in text_list:
        arr.append(t.get_text())
    return " ".join(arr)

In [ ]:
import json

years = ['2021', '2022', '2023']
driver = webdriver.Safari()

dps_results_map = {}
dps_no_results_map = {}
for year in years:
    year_map = {}
    no_year_map = {}
    links = []
    count = -1
    page = 0
    
    # Extract the links from the page
    while count != 0:
        url = f'https://www.dps.texas.gov/news/press-releases?year={year}&field_press_release_type_value=All&page={page}'
        driver.get(url)
        wait = WebDriverWait(driver, 100)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        nsoup = soup.find(class_="views-element-container settings-tray-editable block block-views block-views-blockrecent-press-releases-block-1")
        response = nsoup.find(class_="item-list")
        if response != None:
            links.append(response.find_all('a'))
            count = len(response.find_all('a'))
        else:
            count = 0
        page = page + 1

    for ls in links:
        for link in ls:
            href = link.get('href')
            if href.startswith('/news'):
                # Navigate to the page
                url = f'https://www.dps.texas.gov{href}'
                driver.get(url)

                # Extract the relevant text
                page_soup = BeautifulSoup(driver.page_source, 'html.parser')
                texts = page_soup.find_all('p')
                text = get_text(texts).lower()

                # Store the text in a map
                if "operation lone star" in text:
                    # print(text[0:20])
                    year_map[link.get_text().lower()] = text
                else:
                    # print("none")
                    no_year_map[link.get_text().lower()] = text

    with open(f"archive/DPS/OLS/{year}.json", "w") as outfile:
        # write the dictionary to the file as JSON
        json.dump(year_map, outfile)

    with open(f"archive/DPS/noOLS/{year}.json", "w") as outfile:
        # write the dictionary to the file as JSON
        json.dump(no_year_map, outfile)
    
    dps_results_map[year] = year_map
    dps_no_results_map[year] = no_year_map
            
driver.quit()  

Now lets look at how many articles we obtained

In [12]:
total = 0
for year in dps_results_map:
    total = total + len(dps_results_map[year])
print(total, "total articles containing \"Operation Lone Star\"")

total = 0
for year in dps_no_results_map:
    total = total + len(dps_no_results_map[year])
print(total, "total articles not containing \"Operation Lone Star\"")

37 total articles containing "Operation Lone Star"
220 total articles not containing "Operation Lone Star"


Here is the code for reading the article information from the JSON files, so we can avoid using the web driver every time

In [13]:
import json

years = ['2021', '2022', '2023']

dps_results_map = {}
dps_no_results_map = {}
for year in years:
    
    f = open(f'archive/DPS/OLS/{year}.json')
    data = json.load(f)
    dps_results_map[year] = data

    f = open(f'archive/DPS/noOLS/{year}.json')
    data = json.load(f)
    dps_no_results_map[year] = data
    

## Analysis of DPS articles

In [1]:
import nltk
from nltk import FreqDist

I will convert the maps to numpy arrays for easy handling

In [ ]:
import numpy as np

years = ['2021', '2022', '2023']

dps_results_dict = {}
for year in years:
    # print(results_map[year])
    text_year = " ".join(dps_results_map[year].values())
    dps_results_dict[f'{year}'] = text_year
        
# print(len(all_results_dict))
# print(all_results_dict.items())

dtype = np.dtype([
    ('id', int),
    ('date', object),
    ('text', object),
])

data_list = [(i, k, "".join(list(v))) for i, (k, v) in enumerate(dps_results_dict.items())]

dps_results_np = np.array(data_list, dtype=dtype)

print(dps_results_np[2])

In [ ]:
dps_results_np['date']

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

dps_freqs = {}

for _,d,t in dps_results_np:
    words = tokenizer.tokenize(t)
    filtered_tokens = [token for token in words if token not in stop_words]
    fdist = FreqDist(filtered_tokens)
    dps_freqs[d] = fdist
    
for k, v in dps_freqs.items():
    print(k, ':')
    for word, frequency in v.most_common(10):
        print(f"{word}: {frequency}")
    print()

In [ ]:
year_freqs = {}

temp = []
for _,d,t in all_results_np:
    temp.append(t)
    
year_text = [" ".join(temp[0:12]), " ".join(temp[12:24]), " ".join(temp[24:])]


for i, t in enumerate(year_text):
    words = tokenizer.tokenize(t)
    filtered_tokens = [token for token in words if token not in stop_words]
    fdist = FreqDist(filtered_tokens)
    year_freqs[years[i]] = fdist
    
for k, v in year_freqs.items():
    print(k, ':')
    for word, frequency in v.most_common(10):
        print(f"{word}: {frequency}")
    print()
        

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

because 2023 did not have any articles with our key term, I will not include it in our analysis

In [ ]:
dps_top_words_2021 = dps_freqs['2021'].most_common(30)
dps_top_words_2022 = dps_freqs['2022'].most_common(30)


# Create a bar plot of the most common words
fig, axes = plt.subplots(2,1,figsize=(8,20))
sns.set(style="darkgrid")
sns.barplot(ax = axes[0], 
            x=[w[1] for w in dps_top_words_2021], 
            y=[w[0] for w in dps_top_words_2021])

sns.barplot(ax = axes[1], 
            x=[w[1] for w in dps_top_words_2022], 
            y=[w[0] for w in dps_top_words_2022])

now lets analyze the words overall for the DPS articles

In [ ]:
dps_all_text = " ".join(dps_results_np['text'])
                    
dps_words = tokenizer.tokenize(dps_all_text)
dps_filtered_tokens = [token for token in dps_words if token not in stop_words]
dps_fdist = FreqDist(dps_filtered_tokens)


for word, frequency in dps_fdist.most_common(10):
    print(f"{word}: {frequency}")

In [ ]:
dps_top_words = dps_fdist.most_common(20)

# Create a bar plot of the most common words
sns.set(style="darkgrid")
sns.barplot(x=[w[1] for w in dps_top_words], y=[w[0] for w in dps_top_words])